In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
dataOrg = pd.read_csv('earthquake_dataset.csv')

filtered_data_higher = dataOrg[dataOrg['mag'] >= 5]
filtered_data_lower = dataOrg[dataOrg['mag'] < 5]

result_higher = filtered_data_higher[['mag']]
result_lower = filtered_data_lower[['mag']]

count_higher = filtered_data_higher['mag'].count()
count_lower = filtered_data_lower['mag'].count()

print(result_higher)
print("Number of values with mag >= 5:", count_higher)

print(result_lower)
print("Number of values with mag < 5:", count_lower)

      mag
10    5.0
24    5.8
29    5.3
33    5.2
74    5.0
...   ...
1605  5.0
1616  5.0
1629  5.1
1630  5.2
1632  5.1

[174 rows x 1 columns]
Number of values with mag >= 5: 174
      mag
0     4.4
1     4.2
2     4.1
3     4.4
4     4.5
...   ...
1645  4.4
1646  4.5
1647  4.6
1648  4.7
1649  4.2

[1476 rows x 1 columns]
Number of values with mag < 5: 1476


In [2]:
# Plot map of investigating region
df_geo = gpd.GeoDataFrame(dataOrg, geometry = gpd.points_from_xy(dataOrg.longitude, dataOrg.latitude))
print(df_geo)
world_data = gpd.datasets.get_path('naturalearth_lowres')

                          time  latitude  longitude   depth  mag magType  \
0     2014-09-26T15:38:16.000Z  -20.1000   119.5340  10.000  4.4      mb   
1     2014-10-02T15:52:57.360Z  -10.3478   113.2808  34.820  4.2      mb   
2     2014-10-02T19:46:04.280Z   -9.0761   115.9067  49.760  4.1      mb   
3     2014-10-03T04:57:28.570Z   -9.1953   110.3486  46.440  4.4      mb   
4     2014-10-03T05:26:29.940Z   -9.4264   110.3297  60.700  4.5      mb   
...                        ...       ...        ...     ...  ...     ...   
1645  2024-09-02T14:32:58.275Z   -9.5553   122.0311  65.322  4.4      mb   
1646  2024-09-06T19:58:33.208Z  -32.3238   150.7448   8.069  4.5      mb   
1647  2024-09-13T17:19:49.577Z   -9.2629   110.3796  35.000  4.6      mb   
1648  2024-09-13T21:59:59.504Z   -9.3209   110.3426  10.000  4.7      mb   
1649  2024-09-22T06:16:40.420Z   -9.7413   117.4617  36.431  4.2      mb   

       nst    gap   dmin   rms  ...                                 place  \
0      NaN

In [3]:
# Example tepmplate from assignment file
dataOrg['DateTime'] = pd.to_datetime(dataOrg['time'])
data = dataOrg.sort_values(by='DateTime')
selection1 = (data['DateTime'].dt.year >= 1980) & (data['DateTime'].dt.year < 2010)
selection2 = data['place'].str.contains('netherlands', case=False)
data = data[selection1 & selection2]